In [1]:
#importing the libraries for preprocessing
import pandas as pd
import numpy as np

In [2]:
#creating the dataframe to read the movies.csv file's data
movies_df=pd.read_csv('movies.csv',usecols=['movieId','title'])

In [3]:
#displaying the details of the first 5 rows
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [4]:
#creating a dataframe for reading the ratings.cvs file's data
rating_df=pd.read_csv('ratings.csv',usecols=['userId','movieId','rating'])

In [5]:
#displaying the first five rows of the rating_df dataframe
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [6]:
#Creating a dataframe df and Merging the rating_df dataframe and movies_df dataframe on the basis of movieId as it is common in both of the them. 
df1=pd.merge(rating_df,movies_df,on='movieId')
df1.head(5)

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [7]:
#creating a pivot table as assigning the index as title and columns as userId and we give them the values of rating if present or else be assign zero.
movie_features_df=rating_df.pivot_table(index='movieId',columns='userId',values='rating').fillna(0)
movie_features_df


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
from scipy.sparse import csr_matrix
#the model cannot take values  from table so we create a matrix to take values
movie_features_df_matrix =csr_matrix(movie_features_df.values)

In [9]:
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import process
#the model is NearsestNeighbours and not classifier or a regressor but works the same.

model_knn=NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20)

In [10]:
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [11]:
 def recommender(movie_name,data,model,n_recommendations):
    model.fit(data)
    idx=process.extractOne(movie_name,movies_df['title'])[2]
    print(idx)
    print('Movie Selected: ' ,movies_df['title'][idx],'Index:',idx)
    print("Searching for recommendation.......")
    try:
        distances,indices=model.kneighbors(data[idx],n_neighbors=n_recommendations)
        for i in indices:
            print(movies_df['title'][i].where(i!=idx))
    except:
         print("No Recommendation for this movie")
  
    
    
recommender('liar liar',movie_features_df_matrix,model_knn,20)    
    

1135
Movie Selected:  Liar Liar (1997) Index: 1135
Searching for recommendation.......
1135                                                  NaN
1491          Apple Dumpling Gang Rides Again, The (1979)
4322                                      Wildcats (1986)
3232                                Dr. Dolittle 2 (2001)
2122    Three Days of the Condor (3 Days of the Condor...
4443                        Mouse That Roared, The (1959)
4714                                       Revenge (1990)
4164                             Nicholas Nickleby (2002)
1547                   Muppet Christmas Carol, The (1992)
689                              Gay Divorcee, The (1934)
3891    All About Lily Chou-Chou (Riri Shushu no subet...
4964                                         28 Up (1985)
2468                                    Beach, The (2000)
4515                     Autumn Spring (Babí léto) (2001)
342                        Man of No Importance, A (1994)
1353                                   He G